In [ ]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.backend as K
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import *


In [ ]:
tmp = np.load('../input/bugrep/bug_repair_train.npz')
X = tmp['X']
Y = tmp['Y']
max_len = X.shape[1]

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
def categorical_crossentropyx(trues, preds):
    t = tf.one_hot(trues,depth=50265)
    return tf.metrics.categorical_crossentropy(t,preds)

In [ ]:
with strategy.scope():
    input1 = tfk.layers.Input(shape=(max_len, ), name='input')
    
    x = tfk.layers.Embedding(input_dim=50265, output_dim=512, name='embds')(input1)

    model = tfk.layers.Bidirectional( tfk.layers.GRU(units=1024, return_sequences=True), name='bigru1')(x)
    model = tfk.layers.Bidirectional( tfk.layers.GRU(units=1024, return_sequences=True), name='bigru2')(model)
    model = tfk.layers.Bidirectional( tfk.layers.GRU(units=1024, return_sequences=True), name='bigru3')(model)

    model = tfk.layers.concatenate([model, x], axis=-1, name='con3')

    model = tfk.layers.TimeDistributed( tfk.layers.Dense(2056, activation='relu'), name='td')(model)

    output = tfk.layers.TimeDistributed( tfk.layers.Dense(50265, activation='softmax') ,name='output')(model)

    model = tfk.models.Model(input1, output )
    model.compile(loss=categorical_crossentropyx, metrics=['acc'], optimizer='adam')

    model.summary()

In [ ]:
model.fit(X, Y, verbose=1, batch_size=8, epochs=10)

In [ ]:
model.save_weights('m1.h5')
# model.load_weights('m1.h5')

In [ ]:
tmp = np.load('../input/bugrep/bug_repair_test.npz')
X_ = tmp['X']
Y_ = tmp['Y']


In [ ]:
preds = model.predict(X_[:4],batch_size=4)
preds = np.argmax(preds,axis=-1)
for i in tqdm(range(1,(len(X_)//4)+1)):
    tmp = model.predict(X_[i*4:(i+1)*4],batch_size=4)
    preds = np.append(preds,np.argmax(tmp,axis=-1),axis=0)


In [ ]:
y_t = Y_.flatten()
y_p = np.array(preds).flatten()
y_p = y_p[(y_t!=1)&(y_t!=0)&(y_t!=2)]
y_t = y_t[(y_t!=1)&(y_t!=0)&(y_t!=2)]
print(accuracy_score(y_t,y_p))
